# FFF Vision Using Recursive Function


In [ ]:
#Importing all necessary packages.
import numpy as np
import cv2
print('OpenCV version: '+cv2.__version__)
import pandas as pd
import datetime
import os
import time
import re 

In [ ]:
#used for time calculation.
start = time.perf_counter()
#Reading the folder where the videos are saved.
SRC_FOLDER = r"D:\Pensulvaniya state project\FFF Vision - OpenCV [External]\Vidseos\\"
#reading the vieo_Time_Stamp from device.
df_vidTimes = pd.read_excel(SRC_FOLDER + "Video_Timestamps_1.xlsx")
df_vidTimes.drop(df_vidTimes.columns[0],axis=1,inplace=True)

Image Processing Steps:

Perspective Correction -> Binary Thershold -> Morphological Operations -> Median Blur Operations -> Canny Edge detection ->ROI Selection

In [ ]:
#Function for image perspective correction.
def perspCorrection(img,pt1,pt2,pt3,pt4,scale_width,scale_height):
    
    
    input_pts = np.float32([pt1,pt2,pt3,pt4])
    output_pts = np.float32([[0,0],[scale_width-1,0],[0,scale_height-1],[scale_width-1,scale_height-1]])
    M = cv2.getPerspectiveTransform(input_pts,output_pts)
    #saves the perspective corrected image to a varibale.
    imgPersp = cv2.warpPerspective(img,M,(scale_width, scale_height))
    #grayscale conversion.
    imgGrayPersp = cv2.cvtColor(imgPersp, cv2.COLOR_BGR2GRAY)            
    
    return [imgPersp,imgGrayPersp]


In [ ]:
#function for binary thersholding
def thersold_fun(thersholded):
    thersholded = cv2.threshold(thersholded, 127, 255, cv2.THRESH_BINARY)[1]
    return thersholded

In [ ]:
#function for morphological operations.

def morphOps(medianblr):    
    #erosion and dilation
    mask_erosion = cv2.erode(medianblr, np.ones((8, 0), "uint8"))
    mask_dilation = cv2.dilate(mask_erosion, np.ones((32, 0), "uint8"))
    return mask_dilation

In [ ]:
#function for median bluring.
def medianBlur(imgGrayPersp):

    mblur = cv2.medianBlur(imgGrayPersp, 9)
    return mblur

In [ ]:
#function for canny edge detection
threshold_1 =30
threshold_2 = 80
def canny(cloperation,threshold_1, threshold_2):

    canny_image = cv2.Canny(cloperation,threshold_1, threshold_2) 
    return canny_image

In [ ]:
#extracting ROI from edge detected image.
def extractTopBottom(img,bStart,bEnd):
   
    img_bottom = img[bStart[1]:bEnd[1],bStart[0]:bEnd[0]]      
    
    return img_bottom

In [ ]:
#function for computing extrusion width

def computeW(canny,mm_per_pixel,imgGrayPersp):
    
    indices = np.where(canny != [0])
    LEdge = np.mean(indices[1][::2])
    REdge = np.mean(indices[1][1::2])
    widths = np.abs(REdge-LEdge)
    width_in_mm = mm_per_pixel * widths
    
    text = str(width_in_mm) + ' mm'
    font = cv2.FONT_HERSHEY_SIMPLEX
    org = (50,50)
    fontScale = 1
    color = (255, 0, 0)
    thickness = 2
    img_color = cv2.putText(imgGrayPersp, text, org, font,fontScale, color, thickness)
         
    return [widths, width_in_mm,img_color]

In [ ]:
# Remove # for below code to save the path of folder in which images for analysis is to be saved
imganalysis_path = SRC_FOLDER + "Results/Run-2"

#function for storing images for analysis, need to call this function from recursive function.

def image_for_analysis(imganalysis_path,layers,speeds,imgGrayPersp,thersholded_image,medianblr,cloperation,canny_image,bottom_image,frameCount,imgcolor):
    
    #for storing perspective corrected grayscale images.
    cv2.imwrite(imganalysis_path + '/' + str(layers) + "/" + str(speeds) + "/Gray/" +  "pCorr" + str(frameCount) + ".jpg", imgGrayPersp)
    
    #for storing thersholded images.                       
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/Thresholded Images/" + "thresh" + str(frameCount) + ".jpg", thersholded_image)
    
    #for storing output images of  median blur function.                     
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/median blur/" + "medianblur" + str(frameCount) + ".jpg", medianblr)
    
    #for storing output images of  morphological operations(dilation and erosion).                     
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/Erosion and Dilation/" + "morphOps" + str(frameCount) + ".jpg", cloperation)
    
    #for storing canny images with detected edges.                         
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/Canny/" + "Canny" + str(frameCount) + ".jpg", canny_image)
    
    #for storing the ROI selected images.                        
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/ROI/" + "ROI" + str(frameCount) + ".jpg", bottom_image)    
    
    #for storing the images with labeled width measures.
    cv2.imwrite(imganalysis_path +  '/' + str(layers) + "/" + str(speeds) + "/Vision Measurements/" + "wImg" + str(frameCount) + ".jpg", imgcolor)
    
    return None
#This function returns nothing since it only save images to respective folders in system.
#There should be image folders as mentioned in the save_path

In [ ]:
#function for calculation starting and ending time of videos
def timestamp(layer,speed):

    idx = df_vidTimes.index[(df_vidTimes.Layer==int(layer)) & (df_vidTimes.Speed==int(speed))].to_list()[0]
    start_TS = df_vidTimes.Start_Timestamp[idx]
    end_TS = df_vidTimes.End_Timestamp[idx]
    return[start_TS,end_TS]

In [ ]:
#reading the file names as a list then getting the speed and layer details alone from the video names.
dir_list = os.listdir(SRC_FOLDER)
lst1 = list(map(lambda x: x.replace('.avi', ''), dir_list))
lst1 = list(map(lambda x: x.replace('vid_l', ''), lst1))
lst1 = list(map(lambda x: x.replace('_vR', ''), lst1))
speed_skipp_frames = list(map(lambda x: x.replace('10', '32'), lst1))
speed_skipp_frames = list(map(lambda x: x.replace('20', '20'), speed_skipp_frames))
speed_skipp_frames = list(map(lambda x: x.replace('30', '11'), speed_skipp_frames))
speed_skipp_frames = list(map(lambda x: x.replace('40', '15'), speed_skipp_frames))
speed_skipp_frames = list(map(lambda x: x.replace('50', '13'), speed_skipp_frames))
speed_skipp_frames = list(map(lambda x: x.rsplit("_"), speed_skipp_frames))



In [ ]:
#Main program
num_layers = 20
max_speed = 50
layers = list(range(5,num_layers+1))
speeds = list(range(10,max_speed+10,10))
frame_skip_start = [32,20,11,15,13] 

vidCount = 0
img_debug = False 

w_result_columns=['Layer','vR','Frame','ActualTimestamp','Edge Distance in Pixel','w_Vision']
frame_summary_columns = ['Layer','vR','Start_TS','End_TS','Total_Frames','Per_Frames_Skipped','Skipped_Frames']
lst = []
lst_skip_frames = []
lst_frame_summary = []

pt1 = [192.30,343.00]  
pt2 = [1079.0,379.80]  
pt3 = [153.50,571.90] 
pt4 = [1107.10,611.70] 

scale_width = round(11.7348*200) 
scale_height = round(6.35*200)   

bStart = [655,925]
bEnd = [1300,1270]
fsize =9
threshold_1 =30
threshold_2 = 80
mm_per_pixel = 0.004992138364779874

#Defining the recursive function
def Recursive_function(i):
    
    if(i< len(dir_list)):
        lst = []
        
        vidName = dir_list[i]
        srcVideo = SRC_FOLDER + vidName
        layer = re.search('_l(.*)_v',dir_list[i]).group(1)
        speed = re.search('R_(.*).a',dir_list[i]).group(1)        
        [start_TS,end_TS] = timestamp(layer,speed)
        print('video: {0} starts at {1} and ends at {2}'.format(vidName,start_TS,end_TS))
        cap = cv2.VideoCapture(srcVideo)
        numFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

        if (cap.isOpened() == False):
            print("Error reading video file. Exiting ...")
            exit(0)
        
        frameCount = 0

        while(cap.isOpened()):
            frame_exists, frame = cap.read()
               
               
            if frame_exists:
                
                frameCount = frameCount + 1
                

                if(frameCount >= int(speed_skipp_frames[i][1]) and frameCount <= numFrames - 5): 
                    
                    try:
                                               

                        # call function - correct perspective transform
                        [imgPersp,imgGrayPersp] = perspCorrection(frame,pt1,pt2,pt3,pt4,scale_width,scale_height)
#                         cv2.imshow("image",imgGrayPersp)
#                         cv2.waitKey(0)
                        #thersholding image using thresh.binary function
                        thersholded_image = thersold_fun(imgGrayPersp)
#                         cv2.imshow("image",thersholded_image)
#                         cv2.waitKey(0)
                        #closing operation
                        cloperation = morphOps(thersholded_image)
#                         cv2.imshow("image",cloperation)
#                         cv2.waitKey(0)
                        #medianblur
                        medianblr = medianBlur(cloperation)
                        #Canny detection
                        canny_image= canny(medianblr,threshold_1, threshold_2)
                        #ROI selection
                        bottom_image = extractTopBottom(canny_image,bStart,bEnd)
                        # Extrusion width measurement
                        [bottom_edge_pixels,bottom_edge_dist,img_color] = computeW(bottom_image,mm_per_pixel,imgGrayPersp) 
                                                
                        #Remove Comment line for below code , if images needs to be saved for analysing.
                        #image_for_analysis(imganalysis_path,str(layers[l]),str(speeds[v]),imgGrayPersp,thersholded_image,medianblr,cloperation,canny_image,bottom_image,frameCount,imgcolor)

                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames

                        # Store results in dataframe   
                        lst.append([int(layer),int(speed),frameCount,act_TS,bottom_edge_pixels,bottom_edge_dist])
                        

                        
                    except ValueError as e:
                        
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print(e)
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([int(layer),int(speed),frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])

                    except UnboundLocalError as u:
                        print('Unable to sucessfully process frame {0}, skipping . . .'.format(frameCount))
                        print(u)
                        # Calculate actual timestamp based on excel timestamps and frame number
                        act_TS = start_TS+frameCount*(end_TS-start_TS)/numFrames
                        # Store results in dataframe   
                        lst.append([int(layer),int(speed),frameCount,act_TS,np.nan,np.nan])
                        lst_skip_frames.append([frameCount])

            else:
                
                break       
                   
        cap.release()
        cv2.destroyAllWindows()
        
        print('Finished processing video: {0}'.format(vidName))
        print('')
        print('')   
        results = pd.DataFrame(lst,columns=w_result_columns)
        
        # Save results to csv
        path = SRC_FOLDER + "Results/Run-2/" + 'l' + str(layers[l])+'_vR'+str(speeds[v])+"results1.csv"
        results.to_csv(path)
        
        #appending details for final summary excel.
        lst_frame_summary.append([int(layer),int(speed),start_TS,end_TS,numFrames,len(lst_skip_frames)/numFrames,lst_skip_frames])
        i = i+1
        Recursive_function(i)

In [ ]:
#calling recursive function
i =0
Recursive_function(i)

#converting to Dataframe
frame_summary_results = pd.DataFrame(lst_frame_summary,columns=frame_summary_columns)

# Some more cleanup and data addition
frame_summary_results["Video_Duration"] = frame_summary_results["End_TS"] - frame_summary_results["Start_TS"] 
frame_summary_results["Video_Duration"] = [x.total_seconds() for x in frame_summary_results["Video_Duration"]]
frame_summary_results["FPS"] = frame_summary_results["Total_Frames"]/frame_summary_results["Video_Duration"]
frame_summary_results["Total_Frames_Skipped"] = [len(x) for x in frame_summary_results["Skipped_Frames"]]

# Re-oder columns
frame_summary_results = frame_summary_results[["Layer","vR","Start_TS","End_TS","Video_Duration","Total_Frames","FPS","Total_Frames_Skipped","Per_Frames_Skipped","Skipped_Frames"]]

#Writing to CSV
path_summary = SRC_FOLDER + "Results/Run-2/"+ 'video_processing_summary.csv'
frame_summary_results.to_csv(path_summary)
    
print('Processing of all videos completed successfully!\nSummary results saved at {0}'.format(summary_path))
end = time.perf_counter() - start
print('{:.6f}s for the calculation'.format(end))